# KLダイバージェンスの近似

---

今回はKLダイバージェンスをテイラー展開して，フィッシャー情報量行列で近似するということをやっていきます．

これはどんな時に使われるのかというと，代表例としては[TRPO(Trust Region Policy Optimization)](https://arxiv.org/abs/1502.05477)ですね．

TRPOでは制約付き最適化をするのですが，その時の制約としては以下のようなものです．

$$
D_{K L}\left(\pi_{\theta_{old}} \| \pi_\theta\right) \leq \delta
$$

$\theta$は更新する目的変数で，$\theta_{old}$は更新する前の目的変数の値で，$\sigma$は閾値です．

従来の最適化では，目的変数を$L(\theta)$とするし，制約条件の式を$C(\theta)$とすると，ラグランジュの未定乗数法で次のようにすればいいことを考えます．

$$
L(\theta) - \lambda C(\theta)
$$

上の式を微分して解くのが一般的ですが，今回はパラメータ同士が独立ではないため，解くのがとても難しくなります．

そこでKLダイバージェンスをテイラー展開し，近似することでラグランジュ未定乗数法を使えるようにするのが目的です．

---

早速，KLダイバージェンスをテイラー展開してみましょう．

まずテイラー展開とは次のようなものでしたね．

**テイラー展開**

関数$f(x)$を$x=a$周りで，展開する時のテイラー展開は

$$
f(x)=f(a)+f^{\prime}(a)(x-a)+\frac{f^{\prime \prime}(a)}{2 !}(x-a)^2+\frac{f^{\prime \prime \prime}(a)}{3 !}(x-a)^3+\cdots
$$



これに沿って，KLダイバージェンスのテイラー展開をしてみましょう．

フィッシャ情報量行列を$H$とします．

$$
D_{K L}\left(\pi_{\theta_{old}} \| \pi_\theta\right) \simeq D_{K L}\left(\pi_{\theta_{old}} \| \pi_{\theta_{old}}\right)+\left.\nabla_\theta D_{K L}\left(\pi_{\theta_{old}} \| \pi_\theta\right)\right|_{\theta_{old}} ^{\top}\left(\theta-\theta_{old}\right)+\frac{1}{2}\left(\theta-\theta_{old}\right)^{\top} H\left(\theta-\theta_{old}\right)
$$

右辺の1項目は0になることは分かりますね．

そして，2項目も0になります．

確認してみましょう．

---

**右辺の2項目が0になる理由**

In [1]:
import numpy as np

def KL_divergence(p, q):
    p = np.asarray(p)
    q = np.asarray(q)
    return np.sum(np.where(p != 0, p * np.log2(p / q), 0))

p = np.array([0.3,0.7])
q = np.array([0.2, 0.8])


def fisher_information(p, q):
    n = len(p)
    fisher_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i == j:
                fisher_matrix[i, j] = p[i] / q[i]**2
            else:
                fisher_matrix[i, j] = 0
    
    return fisher_matrix






print(f'フィッシャー情報量行列は{fisher_information(p, q)}')
print("KL divergence: ", KL_divergence(p, q))
print(f'KL divergenceをフィッシャ情報行列で近似した値は{1/2 * ((p-q) @ fisher_information(p, q).T @ (p-q))}')
print(f'それらの差は{KL_divergence(p, q) - 1/2 * ((p-q) @ fisher_information(p, q).T @ (p-q))}')

フィッシャー情報量行列は[[7.5     0.     ]
 [0.      1.09375]]
KL divergence:  0.04063719565666954
KL divergenceをフィッシャ情報行列で近似した値は0.042968749999999986
それらの差は-0.002331554343330447


うまく近似できてそうですね．
